In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")
sys.path.append("../")
import os
print(os.getcwd())
from models.basic import CNN
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# width_to_output_mapping = {}
# device = "cuda"

# cnn = CNN(nc=1, cnn_type="default64", first_conv_opts=None)
# cnn.to(device)
# for width in range(233,240):
#     try:
#         t = torch.zeros(1, 1, 32, width).to(device)
#     except:
#         device = "cpu"
#         t = torch.zeros(1, 1, 32, width).to(device)

#     shape = cnn(t).shape
#     width_to_output_mapping[width] = shape[0]
    

/media/data/GitHub/simple_hwr/hwr_utils/testing


In [5]:
import tempfile
from pathlib import Path
import numpy as np
from torch.utils.data import DataLoader
from models.basic import CNN, BidirectionalRNN
from torch import nn
from loss_module.stroke_recovery_loss import StrokeLoss
import torch
from models.CoordConv import CoordConv
from trainers import TrainerStrokeRecovery
from hwr_utils.stroke_dataset import StrokeRecoveryDataset, read_img
from hwr_utils.stroke_recovery import *
from hwr_utils import utils, stroke_recovery
from torch.optim import lr_scheduler
from timeit import default_timer as timer
from train_stroke_recovery import graph
from hwr_utils.distortions import *
from hwr_utils.stroke_plotting import *

## DRAW THE IMAGE
#Pillow-simd

from PIL import Image, ImageDraw

def loader():
    folder = Path("online_coordinate_data/3_stroke_vSmall")
    folder = Path("online_coordinate_data/8_stroke_vSmall_16")
    folder = Path("online_coordinate_data/MAX_stroke_vTEST_AUGMENTFull")

    print(os.getcwd())
    x_relative_positions = True
    test_size = 5
    train_size = None
    batch_size=32
    #gt_format = ["x","y","stroke_number","eos"]
    gt_format = ["x","y","sos","eos"]

    test_dataset=StrokeRecoveryDataset([folder / "test_online_coords.json"],
                            img_height = 61,
                            num_of_channels = 1.,
                            max_images_to_load = test_size,
                            root=r"../../data",
                            cnn=None,
                            gt_format=gt_format,
                            )

    test_dataloader = DataLoader(test_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=3,
                                  collate_fn=test_dataset.collate,
                                  pin_memory=False)

    device="cuda"
    globals().update(locals())
    return test_dataloader

def reload_and_get_new_example():
    test_dataloader = loader()
    return next(iter(test_dataloader))

def get_instance():
#     if not 'test_dataloader' in globals():
#         test_dataloader = loader()
    example = next(iter(test_dataloader)) # BATCH, WIDTH, VOCAB
    instance = example["gt_list"][0].numpy()
    return instance, example

def test_drawing():
    instance, example = get_instance()
    
    draw_from_gt(instance)
    draw_from_raw(gt_to_raw(instance))

loader()
#test_drawing()

# example = next(iter(test_dataloader)) # BATCH, WIDTH, VOCAB
# instance = example["gt_list"][0].numpy()
instance, example = get_instance()



/media/data/GitHub/simple_hwr/hwr_utils/testing
../../data/online_coordinate_data/MAX_stroke_vTEST_AUGMENTFull/test_online_coords.json


INFO:root.hwr_utils.stroke_dataset:('Original dataloader size', 501)
INFO:root.hwr_utils.stroke_dataset:('Dataloader size', 5)


UNK NUMBER OF SAMPLES!!!
UNK NUMBER OF SAMPLES!!!
UNK NUMBER OF SAMPLES!!!
UNK NUMBER OF SAMPLES!!!
UNK NUMBER OF SAMPLES!!!


In [9]:
strokes = 0
stroke_points = 0
for item in test_dataloader:
    strokes += np.sum([torch.sum(i[:,2]) for i in item["gt_list"]])
    stroke_points += np.sum([i.shape[0] for i in item["gt_list"]])

print(strokes)
print(stroke_points)
print(stroke_points/strokes)

171.0
2947
17.23391812865497


In [29]:
"1".split(",")

['1']

In [51]:
# PLOT VS PLOT WITH PAD

#instance = next(iter(test_dataset))["gt"].transpose(1,0)
#instance, example = get_instance()


# print(instance.shape)
# get_x_y_min_max_from_gt(instance)
# img = draw_from_gt(instance, show=True)

new_gt = random_pad(instance, 15, 15)
draw_from_gt(new_gt, show=True, right_padding="random", color=(0,0,0), max_width=12).shape


(61, 410, 3)

In [ ]:
draw_from_gt(new_gt, show=True, right_padding="random", linewidth=4, color=(0,0,0))#max_width=12)


In [ ]:
from time import sleep
device="cuda"
example = next(iter(test_dataloader)) # BATCH, WIDTH, VOCAB
output = Path("./TEST_GRAPHING")
print(example.keys())

# This graphs the directions etc.
# graph(example, save_folder=output, x_relative_positions=x_relative_positions)

def draw_temp(stroke_list):
    fd, path = tempfile.mkstemp()
    path = "./temp"
    path += ".png"
    ratio = None
    try:
        draw_strokes(stroke_list, ratio, save_path=path, line_width=.8)
        img = read_img(path)
    finally:
        #os.remove(path)
        pass
    return img


In [ ]:
example = next(iter(test_dataloader)) # BATCH, WIDTH, VOCAB
instance = example["gt_list"][0].numpy()
start_points = stroke_recovery.relativefy(instance[:,2])
s = start_points
start_indices = np.argwhere(s==1).astype(int).reshape(-1)
l = np.split(instance[:,0:2], start_indices)

output = []

for item in l:
    if item.shape[0]:
        output.append({"x": item[:,0].tolist(), "y": item[:,1].tolist()})

#print(output)
draw_strokes(output, x_to_y=None)

img = draw_temp(output)
print(img.shape)

In [ ]:
path = "./temp"
img = read_img(path)


In [ ]:
import dill
dill.dump_session('notebook_env.db')
#dill.load_session('notebook_env.db')

print(output)

In [ ]:
print(example.keys())

In [ ]:
# Test draw
img  = Image.new("L", (50, 60), 255)
draw = ImageDraw.Draw(img)
draw.line([(0,0), (50,50)])
img.show()

In [ ]:
warp_points


In [ ]:
x = np.random.randint(5,10,(10,10))

In [ ]:
top = 1
bottom = 4
left = 2
right = 3

np.pad(x, ((top,bottom),(left,right)))


In [ ]:
y = x[0]+1

In [ ]:
print(y)

In [ ]:
print(x)

In [ ]:
instance, example = get_instance()

print(instance.shape)
img = draw_from_gt(instance, show=False)
new_gt, new_img = random_pad(instance, img, 20, 20)

img = Image.fromarray(new_img, 'L')
img.show()
draw_from_gt(new_gt, show=True)


In [ ]:
img = draw_from_gt(instance, show=True)
img = blur(img.copy() / 127.5 - 1.0, max_intensity=1.1)
print(img)
img = (img + 1.0) * 127.5
f = Image.fromarray(np.uint8(img), 'L')
f.show()



In [ ]:
from hwr_utils.grid_distortion import *
img = draw_from_gt(instance, show=True)
img = Image.fromarray(img, 'L')
img.show()

if False:
    img = warp_points(instance, random_state=None)
    img = np.c_[img, instance[:,2]]


In [19]:
## DISTORT IMAGE
from hwr_utils.distortions  import *
from hwr_utils.stroke_plotting  import *

instance = instance[:,:2]
img_prime = draw_from_gt(instance, show=False, max_width=3)
for i in range(0,1):
    img = img_prime.copy()
    if True:
        img = gaussian_noise(
                blur(
                    random_distortions(img_prime.copy()),
                    max_intensity=1.3),
                max_intensity=.1)
    f = Image.fromarray(np.uint8(img), 'L')
    #f.show()
# Render GT on top of image
f = f.convert("RGB")
img_prime = draw_from_gt(instance, show=False, linewidth=1, color=[255,0,0], alpha=True) # RED
gt = Image.fromarray(np.uint8(img_prime), 'RGBA')
f.paste(gt, (0, 0), gt)
f.show()

#f = Image.fromarray(np.uint8(img), 'L')
    


In [9]:
instance2 = instance

In [ ]:
from matplotlib import pyplot as plt
l = []
max_width = 3
for i in range(1000):
    l.append(max(int(abs(np.random.randn())*(max_width-1)*.5 + 1 ),1))

plt.hist(l)

In [ ]:
### WARP

for i in range(10):
    gt = instance.copy()
    height = 61
    x = warp_points(gt*height)/height # convert to pixel space
    x = np.c_[x, instance[:,2]]
    img = draw_from_gt(x, show=True, max_width=2, save_path="./warp.png")


In [ ]:
m = np.array([[0,2,3,4,5],[0,2,3,4,5]])
m = m.reshape(-1,2)
print(m.shape)
x = warp_points(m)
print(x)

In [ ]:
f.show()



In [ ]:
im = Image.fromarray(np.uint8(img), 'L')
transparent_area = (0,0,50,50)

mask=Image.new('L', im.size, color=255)
draw=ImageDraw.Draw(mask) 
draw.rectangle(transparent_area, fill=0)
im.putalpha(mask)
im.show()

In [ ]:
def show(numpy_array):
    f = Image.fromarray(np.uint8(numpy_array), 'L')
    f.show()
    return f

img_prime = draw_from_gt(instance, show=False, max_width=3)
background = Image.fromarray(np.uint8(img_prime), 'L')

for i in range(0,10):
    img = img_prime.copy()

    if False:
        img = random_distortions(img_prime.copy(), sigma=20.0, noise_max=2)
    else:
        img = blur(img_prime.copy(), max_intensity=1.0)
    
    show(img)
    # OVERLAY
    #bg = overlay_images(background_img=img, foreground_gt=instance, normalized=False)
    #bg.show()


In [ ]:
example = reload_and_get_new_example()

In [ ]:
print(example.keys())
print(example["paths"])

def show_from_processed_img(img, redo=True):
    if redo:
        img = (img[:,:,0] + 1) * 127.5
    print(img.shape)
    img = Image.fromarray(np.uint8(img), 'L')
    img.show()

# Draw from Path
path = example["paths"][0]
old_img = read_img(path)
show_from_processed_img(old_img)

gt = example["gt_list"][0]
if False:
    ## Draw from GT
    new_img = draw_from_gt(gt, show=True, max_width=2)
    print(gt.shape, new_img.shape)

#show_from_processed_img(new_img, redo=False)


In [ ]:
# Prep image, try to match the one drawn from path in size
print(len(gt))
new_img = StrokeRecoveryDataset.prep_image(gt.numpy(), img_height=61)[::-1]
show_from_processed_img(new_img)


In [ ]:
x = old_img[np.newaxis].transpose(0,3,1,2)
print(x.shape)
f = cnn(Tensor(x).to("cuda"))
f.shape

In [ ]:
5, 1, 61, 1126
5, 1, 60, 1236
print(example["gt_list"][0].shape)

In [ ]:
distance_metric = lambda x, y: np.append(((x[:-1] - x[1:]) ** 2 + (y[:-1] - y[1:]) ** 2) ** (1 / 2), 0)
distance_metric(np.array([1,2,3]),np.array([2,3,5]))

In [ ]:
from hwr_utils.stroke_recovery import calc_stroke_distances,parameterize_as_func_of_distance

calc_stroke_distances([1,2,3,4],[2,3,4,5],[1,0,1,0])

In [ ]:
parameterize_as_func_of_distance([1,2,3,4],[2,3,4,5],[1,0,1,0])

In [ ]:
for image_width in width_to_output_mapping.keys():
    print(image_width, width_to_output_mapping[image_width])

In [ ]:
width_to_output_mapping = lambda width: -(width % 2) + width + 4
inverse = lambda gt_len: gt_len - 3

In [ ]:
for i in range(10,20): 
    m = width_to_output_mapping(i)
    print("forward", i,m)
    print(inverse(m))
    

In [ ]:
int(65 / 4) + 1

In [44]:
input_tensor = Tensor(4,1,10,20)
batch_size, _, x_dim, y_dim = input_tensor.size()

xx_channel = torch.arange(x_dim).repeat(1, y_dim, 1)
yy_channel = torch.arange(y_dim).repeat(1, x_dim, 1).transpose(1, 2)

# Rescale from 0 to 1
xx_channel = xx_channel.float() / (x_dim - 1)
yy_channel = yy_channel.float() / (y_dim - 1)


if True: # Create default CoordConv, and one scaled to be same scale as y CoordConv
    xx_rec_channel = xx_channel * x_dim / y_dim

    xx_channel = xx_channel.repeat(batch_size, 1, 1, 1).transpose(2, 3)
    yy_channel = yy_channel.repeat(batch_size, 1, 1, 1).transpose(2, 3)
    xx_rec_channel = xx_rec_channel.repeat(batch_size, 1, 1, 1).transpose(2, 3)

    ret = torch.cat([
        input_tensor,
        xx_channel.type_as(input_tensor),
        xx_rec_channel.type_as(input_tensor),
        yy_channel.type_as(input_tensor)], dim=1)

    if True:
        xx_sine_channel = torch.sin(xx_rec_channel*4)
        ret = torch.cat([ret, xx_sine_channel.type_as(input_tensor)], dim=1)

In [45]:
ret.shape

torch.Size([4, 5, 10, 20])

In [46]:
xx_sine_channel.shape

torch.Size([4, 1, 10, 20])

In [49]:
#print(xx_rec_channel*4)
print(xx_channel)

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111,
           0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111,
           0.1111, 0.1111, 0.1111, 0.1111],
          [0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222,
           0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222, 0.2222,
           0.2222, 0.2222, 0.2222, 0.2222],
          [0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333,
           0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333, 0.3333,
           0.3333, 0.3333, 0.3333, 0.3333],
          [0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444,
           0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444, 0.4444,
           0.4444, 0.4444, 0.4444, 0.4444],
          [0.5556, 0.5556, 0.5

In [50]:
print(xx_sine_channel)

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.2204, 0.2204, 0.2204, 0.2204, 0.2204, 0.2204, 0.2204, 0.2204,
           0.2204, 0.2204, 0.2204, 0.2204, 0.2204, 0.2204, 0.2204, 0.2204,
           0.2204, 0.2204, 0.2204, 0.2204],
          [0.4300, 0.4300, 0.4300, 0.4300, 0.4300, 0.4300, 0.4300, 0.4300,
           0.4300, 0.4300, 0.4300, 0.4300, 0.4300, 0.4300, 0.4300, 0.4300,
           0.4300, 0.4300, 0.4300, 0.4300],
          [0.6184, 0.6184, 0.6184, 0.6184, 0.6184, 0.6184, 0.6184, 0.6184,
           0.6184, 0.6184, 0.6184, 0.6184, 0.6184, 0.6184, 0.6184, 0.6184,
           0.6184, 0.6184, 0.6184, 0.6184],
          [0.7764, 0.7764, 0.7764, 0.7764, 0.7764, 0.7764, 0.7764, 0.7764,
           0.7764, 0.7764, 0.7764, 0.7764, 0.7764, 0.7764, 0.7764, 0.7764,
           0.7764, 0.7764, 0.7764, 0.7764],
          [0.8962, 0.8962, 0.8